In [1]:
with open('./input/jaychou_lyrics.txt','r',encoding='utf-8') as fh:
    lines = fh.read()
    print(len(lines))

64925


In [2]:
content = lines.replace('\n', ' ').replace('\r', ' ')

In [3]:
content[0:100]

'想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里 我每天每天每天在想想想想著你 这样的甜蜜 让我开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的让我心疼的可'

In [4]:
# construct data iterator 
content = content[0:20000]
idx_to_char = list(set(content))
char_to_idx = dict([(char,int(i)) for i,char in enumerate(idx_to_char)])
vocab_size = len(idx_to_char)

In [5]:
vocab_size

1465

In [6]:
idx_corpus = [char_to_idx[c] for c in content]

In [7]:
s = ''.join([idx_to_char[i] for i in idx_corpus])
print(s[0:40])

想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里 我每天每天每


In [8]:
import random
import mxnet as mx
import sys
sys.path.append('..')
from util import update

def data_iter_random(num_steps,batch_size,corpus,ctx=mx.cpu()):
    #随机生成 data iterator，用yield返回？？
    #先随机分段
    #再随机选择batch_size个字段
    len_corpus=len(corpus)
    num_patch = len_corpus//num_steps
    num_batch = num_patch//batch_size 
    idx_list = [x for x in range(num_patch)]
    random.shuffle(idx_list)
    
    for batch in range(num_batch):
        #在idx_list中随机选择batch_size 个元素，选完之后在idx_list中删除这些元素
        to_use = random.sample(idx_list,batch_size)
        batch_data = []
        batch_label = []
        for item in to_use:
            idx_list.remove(item)
        for step in to_use:
            #i * num_patch ~ (i+1)*num_path
            data = corpus[step*num_steps:(step+1)*num_steps]
            batch_data.append(data)
            #判断最后一个是否越界
            if (step+1)*num_steps < len_corpus:
                label = corpus[step*num_steps + 1:(step+1)*num_steps+1]
            else:
                label = corpus[step*num_steps+1:(step+1)*num_steps]
                label.append('0')#填充字符
            batch_label.append(label)
        #聚合
        #ndarray batch_size * num_steps
        batch_data = mx.ndarray.array(batch_data)
        batch_label = mx.ndarray.array(batch_label)
        yield(batch_data,batch_label)
    

In [9]:

data_iter = data_iter_random(3,10,idx_corpus)

In [10]:
def data_iter_consective(num_steps,batch_size,corpus,ctx=mx.cpu()):
    #连续采样
    num_patch = len(corpus)//num_steps
    num_batch = num_patch//batch_size
    
    
    for i_batch in range(num_batch):
        total_data = []
        total_label = []
        for i_batch_size in range(batch_size):
            start_pos = i_batch+num_batch*(i_batch_size)
            data = corpus[start_pos*num_steps:(start_pos+1)*num_steps]
            #判断最后是否越界
            if(start_pos+1)*num_steps < len(corpus):
                label = corpus[start_pos*num_steps+1:(start_pos+1)*num_steps+1]
            else:
                label = corpus[start_pos*num_steps+1:(start_pos+1)*num_steps]
                label.append('0')
            total_data.append(data)
            total_label.append(label)
        total_data = mx.ndarray.array(total_data)
        total_label = mx.ndarray.array(total_label)
        yield(total_data,total_label)
            

In [11]:
#验证 iter是否正确
def valid_data_iter():
    corpus = list(range(10000))
    batch_size = 10
    num_steps = 3
    data_iter_rand = data_iter_consective(batch_size=batch_size,num_steps=num_steps,corpus=corpus)
    count = 0
    for item in data_iter_rand:
        if(count>10):
            break;
        count +=1
        print(item)

In [12]:
#数据还需要编码，采用最简单的one-hot-encoding 编码

In [13]:
import mxnet.ndarray as nd
for data,label in data_iter_consective(3,10,idx_corpus):
    inp =[nd.one_hot(x,vocab_size) for x in data.T]
    break
    
def one_hot(data,vocab_size):
    inp = [nd.one_hot(x,vocab_size) for x in data.T]
    #len(inp) = num_steps
    return inp



In [14]:
#尝试使用gpu
import sys
sys.path.append('..')
import util
ctx=util.try_gpu()

#construct rnn
hidden_dim = 256
output_dim = vocab_size
input_dim = vocab_size
batch_size  = 32
std = .01
learning_rate = 0.2
def get_params():
    #set params
    W_xh= nd.random_normal(scale=std,shape=(input_dim,hidden_dim),ctx=ctx)
    W_hh = nd.random_normal(scale=std,shape=(hidden_dim,hidden_dim),ctx=ctx)
    b_h = nd.zeros(hidden_dim,ctx=ctx)
    
    W_hy = nd.random_normal(scale = std,shape=(hidden_dim,output_dim),ctx=ctx)
    b_y = nd.zeros(output_dim,ctx=ctx)
    params = [W_xh,W_hh,b_h,W_hy,b_y]
    for param in params:
        param.attach_grad()
    return params


In [15]:
def rnn(inputs,states,*params):
    #inputs  is a list shape = batch_size * input_dim
    H = states
    W_xh,W_hh,b_h,W_hy,b_y = params
    outputs = [] 
    for X in inputs:
        H = nd.tanh(nd.dot(X,W_xh)+nd.dot(H,W_hh)+b_h)
        Y = nd.dot(H,W_hy)+b_y
        outputs.append(Y)
    return (outputs,H)
        
#test
params = get_params()
states = nd.zeros((batch_size,hidden_dim),ctx=ctx)
inputs = [nd.ones((batch_size,input_dim),ctx=ctx)] *3
(outputs,H) = rnn(inputs,states,*params)
print('output len:%d'%len(outputs))
print('output[0] shape',outputs[0].shape)

output len:3
output[0] shape (32, 1465)


In [16]:
#training and inference

def predict_rnn(rnn,prefix,vocab_size,num_char,params,hidden_dim,char_to_idx,idx_to_char):
    H = nd.zeros(shape=(1,hidden_dim),ctx=ctx)
    outputs = []
    outputs.append(char_to_idx[prefix[0]])
    
    for i in range(num_char+len(prefix)):
        x = nd.array([outputs[-1]],ctx=ctx)
        x = one_hot(x,vocab_size)
        out, H = rnn(x,H,*params)
        if i< len(prefix)-1:
            new_input = char_to_idx[prefix[i+1]]
        else:
            new_input  = int(out[-1].argmax(axis=1).asscalar())
        outputs.append(new_input)
    return ''.join([idx_to_char[x] for x in outputs])
    
#prefix='书由 nainia'
#predict_rnn(rnn,prefix,vocab_size,10,params,hidden_dim,char_to_idx,idx_to_char)



In [17]:
# gradient cliping
def grad_clipping(params,theta):
    #python传参是传的引用，所有不用返回值
    norm = nd.array([0.0],ctx=ctx)
    for p in params:
        norm = norm + nd.sum(p.grad**2)
    norm = nd.sqrt(norm).asscalar()
    if norm > theta:
        for p in params:
            p.grad[:] =p.grad[:]* theta/norm
    

In [18]:
# training
#perplexity 困惑度
import numpy as np
def train_and_predict(num_steps):
    #how?
    #定义损失
    seq1 = '分开'
    seq2 = '不分开'
    seq3 = '战争中部队'
    seqs = [seq1, seq2, seq3]
    loss = mx.gluon.loss.SoftmaxCrossEntropyLoss()
    
    #生成数据迭代器
    #循环num_epoch
    num_epochs = 200
    
    params = get_params()
    for epoch in range(num_epochs):
        train_loss = 0.0
        data_iter = data_iter_random(num_steps,batch_size,idx_corpus)
        num_example = 0.0
        for data,label in data_iter:
            data = one_hot(nd.array(data,ctx=ctx),vocab_size)
            H = nd.zeros(shape=(batch_size,hidden_dim),ctx=ctx)
            label = nd.array(label,ctx=ctx)
            with mx.autograd.record():
                outputs,H = rnn(data,H,*params)
                #计算loss
                #for i in range(len(out)):
                #    myloss = loss(out[i],label[:,i])
                #    train_loss += nd.sum(myloss).asscalar()
                label = label.T.reshape((-1,))
                outputs = nd.concat(*outputs, dim=0)
                myloss=loss(outputs,label)
            myloss.backward()
            grad_clipping(params,5)
            update(params,learning_rate)
            train_loss += nd.sum(myloss).asscalar()
            num_example += myloss.size
        print('Epoch:%d  Perplexity:%f'%(epoch,np.exp(train_loss/num_example)))
        if epoch % 10 ==0:
            #predict_rnn
            for seq in seqs:
                print(predict_rnn(rnn,seq,vocab_size,10,params,hidden_dim,char_to_idx,idx_to_char))
            
    
    
    
        

In [19]:
num_steps=35
train_and_predict(num_steps=num_steps)

Epoch:0  Perplexity:892.700149
分开           
不分开           
战争中部队           
Epoch:1  Perplexity:483.236306
Epoch:2  Perplexity:428.574377
Epoch:3  Perplexity:410.862956
Epoch:4  Perplexity:390.061948
Epoch:5  Perplexity:378.325043
Epoch:6  Perplexity:363.259748
Epoch:7  Perplexity:356.755914
Epoch:8  Perplexity:337.454115
Epoch:9  Perplexity:328.284121
Epoch:10  Perplexity:311.815427
分开 我 我 我 我 我 
不分开 我 我 我 我 我 
战争中部队 我 我 我 我 我 
Epoch:11  Perplexity:299.550010
Epoch:12  Perplexity:290.568963
Epoch:13  Perplexity:278.737281
Epoch:14  Perplexity:264.593851
Epoch:15  Perplexity:256.465755
Epoch:16  Perplexity:246.065591
Epoch:17  Perplexity:239.134919
Epoch:18  Perplexity:228.973926
Epoch:19  Perplexity:218.598968
Epoch:20  Perplexity:208.192696
分开 你不的让我不 你想的
不分开 你不的让我不 你想的
战争中部队 你不的让我不 你想的
Epoch:21  Perplexity:200.446446
Epoch:22  Perplexity:190.808660
Epoch:23  Perplexity:183.945792
Epoch:24  Perplexity:171.890856
Epoch:25  Perplexity:165.144290
Epoch:26  Perplexity:157.784655
Epoch:27

In [20]:
import mxnet.ndarray as nd

In [21]:
a = nd.ones((10,3))

In [22]:
a


[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
<NDArray 10x3 @cpu(0)>

In [23]:
a.T.reshape((-1,))


[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
<NDArray 30 @cpu(0)>